# TME 3 - Régularisation L1

In [64]:
import numpy as np
from sklearn import datasets
from sklearn import metrics
from sklearn.utils import shuffle


cancer = datasets.load_breast_cancer()
X = cancer.data
print("shape of X:", X.shape)
y = cancer.target
print("shape of Y:", y.shape)
print("classes in Y: ", np.unique(y))
X, y = shuffle(X, y)



shape of X: (569, 30)
shape of Y: (569,)
classes in Y:  [0 1]


In [72]:
from scipy.optimize import approx_fprime

def loss(theta, X, y, pen):
    (l, n) = X.shape
    s = 0
    for i in range(l):
        s += (y[i] - theta.dot(X[i]))**2 + pen * np.linalg.norm(theta, 1)
    return 1./l * s

def gradient_l1(theta, X, y, max_iter=1000, step=1e-1, pen=1, verbose=0):
    """ Performs gradient clipping to find the best theta
    :param:
    
    :return:"""
    
    (l, n) = X.shape
    for it in range(max_iter):
        if verbose >= 1:
            print(it, ":")
        for i in range(l):
            idx = np.random.randint(0, l)
            local_loss = lambda theta: loss(theta, np.array([X[idx]]), np.array([y[idx]]), pen)
            grad = approx_fprime(theta, local_loss, epsilon=step)
            if verbose >= 2:
                print(grad)
            
#             print("Theta:", theta)
            theta_prime = theta - step * grad
            
#             print("Theta_prime:", theta_prime)
            theta_prime[theta * theta_prime < 0] = 0
            theta = theta_prime
            
        if verbose >= 1:
            general_loss = loss(theta, X, y, pen)
            print("L = %f" % (general_loss))
    return theta

t = gradient_l1(np.ones_like(X[0]), X, y, max_iter=150, step=1e-2, verbose=1)

0 :
L = 472515664136208182696662760606379540480.000000
1 :
L = 472515664136208182696662760606379540480.000000
2 :
L = 472515664136208182696662760606379540480.000000
3 :
L = 1254155795535993402234843149397624570412324093952.000000
4 :
L = 1254155795535993402234843149397624570412324093952.000000
5 :
L = 1254155795535993402234843149397624570412324093952.000000
6 :
L = 1254155795535993402234843149397624570412324093952.000000
7 :
L = 1254155795535993402234843149397624570412324093952.000000
8 :
L = 1254155795535993402234843149397624570412324093952.000000
9 :
L = 1254155795535993402234843149397624570412324093952.000000
10 :
L = 1254155795535993402234843149397624570412324093952.000000
11 :
L = 1254155795535993402234843149397624570412324093952.000000
12 :
L = 1254155795535993402234843149397624570412324093952.000000
13 :
L = 1254155795535993402234843149397624570412324093952.000000
14 :
L = 1254155795535993402234843149397624570412324093952.000000
15 :
L = 12541557955359934022348431493976245704123

KeyboardInterrupt: 

In [59]:
from sklearn.base import BaseEstimator
class LinearClassifier(BaseEstimator):
    def __init__(self, max_iter=150, step=1e-2, pen=1, verbose=1):
        self.theta= 0
        self.max_iter = max_iter
        self.step = step
        self.pen = pen
        self.verbose = verbose
    
    def fit(self, x, y):
        self.theta = gradient_l1(np.ones_like(x[0]), x, y, 
                                 max_iter=self.max_iter, 
                                 step=self.step, 
                                 pen=self.pen,
                                 verbose=self.verbose)
        
    def predict(self, x):
        for x_ex in x:
            pred= np.sign([self.theta.dot(x_ex) for x_ex in x])/2+0.5
            print ("Sortie Predite",pred)
            
        return pred

    def get_theta(self):
        return self.theta

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import KFold



kf = KFold(n_splits=4)


for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = y[train_index], y[test_index]
    clf = LinearClassifier(max_iter=50, step=1e-3, verbose=0)
    clf.fit(X_train, Y_train)
    pred = clf.predict(X_test)
    
    print("KFold:")
    print("theta : ", clf.get_theta())
    print("Y_Pred: ", pred)
    print("Y_reel: ", Y_test)
    print("Score: ", accuracy_score(pred, Y_test))
    print("Errors: ", Y_test[pred != Y_test])


Sortie Predite [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Sortie Predite [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.